In [1]:
import django
import os
import sys
import shutil

path='../../'

source_file_path = f'{path}.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

from dotenv import dotenv_values

_config = dotenv_values(".env")
_username = _config["username"]

session_file_path = f'{path}{_username}.session'

shutil.copy(session_file_path, destination_file_path)


project_path = path  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()


In [2]:
from abc import ABC, abstractmethod
from typing import Optional
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.sync import TelegramClient
from dotenv import dotenv_values
from telethon.tl.types import Message, PeerChannel
from datetime import datetime, timezone
from tqdm import tqdm
from django.forms.models import model_to_dict
from PostAnalyzer.models import (
    Market,
    Post,
    Predict,
    Symbol,
    PositionSide,
    MarginMode,
    SettingConfig,
    Channel
)
from Shared.findError import findError
from Shared.errorSaver import errorSaver
from Shared.helpers import print_colored, find_nearest_number_for_coienex_leverage, convertToJsonFile
from Shared.Constant import PositionSideValues, MarketValues, MAX_PROFIT_VALUE, OrderSide, OrderType, MarginModeValues
import time
from Shared.Exchange import exchange

In [3]:
_config = dotenv_values(".env")

_api_id = _config["api_id"]
_api_hash = _config["api_hash"]
_username = _config["username"]

In [4]:
# client: TelegramClient = await TelegramClient(_username, _api_id, _api_hash).start()
# peer_channel =  PeerChannel(2101974116)
# telegram_channel = await client.get_entity(peer_channel)

# offset_id = 0
# limit = 100
# all_messages: list[Message] = []
# total_messages = 0
# total_count_limit = 0

# end_date = datetime(2024, 12, 7, tzinfo=timezone.utc)

# shouldStop = False
# while not shouldStop:
#     print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
#     history = await client(
#     GetHistoryRequest(
#             peer=telegram_channel,
#             offset_id=offset_id,
#             # timestamp not milli timestamp
#             offset_date=None,
#             add_offset=0,
#             limit=limit,
#             max_id=0,
#             min_id=0,
#             hash=0,
#         )
#     )

#     if not history.messages:
#         break

#     messages: list[Message] = history.messages
#     for message in tqdm(messages):
#         message_date = message.date.replace(tzinfo=timezone.utc)
#         # if start_date < message_date:
#         #     continue

#         if message_date < end_date:
#             shouldStop = True
#             break
#         all_messages.append(message.to_dict())

#     offset_id = messages[len(messages) - 1].id
#     total_messages = len(all_messages)
#     # if total_count_limit != 0 and total_messages >= total_count_limit:
#     #     break

# await client.disconnect()

# convertToJsonFile(all_messages, 'aaa', '.')



In [5]:
from Channels.KingCryptoChannel import KingCryptoChannel
from Shared.helpers import print_colored, returnSearchValue
import re


king = KingCryptoChannel()

In [6]:
msg =  "💎BRETT/USDT \n🔴SHORT\n\n➡️Entry1: 0.21806\n➡️Entry2: 0.22138\n\n\n⛳️Targets : \n\n⛳️Target 1 : 0.21456\n⛳️Target 2 : 0.21238\n⛳️Target 3 : 0.20974\n\n🚫 Stop : 0.22534\n\n📊LV : 12X\n\n‼️3%  of wallet future\n#_کینگ‌کریپتو\n\nVipbaz1"
msg1 = "💎#CHR/USDT \n🟢LONG\n\n➡️ Entry1: Market\n➡️ Entry2: 0.336\n\n⛳️Target :0.348_0.353_0.362\n\n\n🚫 Stop :0.332\n\n📊LV : 15x\n\n‼️3% of wallet future\n#_کینگ‌کریپتو"
msg2 =  "🔼DOGE/USDT \n🟢LONG\n\n➡️Entry1: 0.16461\n➡️Entry2: 0.16203\n\n\n⛳️Targets : \n\n⛳️Target 1 : 0.16685\n⛳️Target 2 : 0.16838\n⛳️Target 3 : 0.17028\n\n🚫 Stop : 0.16076\n\n📊LV : 15X\n\n‼️3%  of wallet future\n#_کینگ‌کریپتو\n\nVipbaz1"
msg3 = "🔼#Btc/USDT \n🟢LONG\n\n➡️Entry1: Market\n➡️Entry2: 60532\n\n\n⛳️Targets : \n\n⛳️Target 1 : 62508\n⛳️Target 2 : 63610\n⛳️Target 3 : 65033\n\n🚫 Stop : 59201\n\n📊LV : 15X"  
msg4 = "🔼#Gala/USDT \n\n🟢LONG\n\n➡️Entry1: Market\n➡️Entry2: 0.02601\n\n\n⛳️Targets : \n\n⛳️Target 1 : 0.02758\n⛳️Target 2 : 0.02800\n⛳️Target 3 : 0.02853\n⛳️Target 4 : 0.02953\n⛳️Target 5 : 0.03074\n\n🚫 Stop : 0.02513\n\n📊LV : 8X\n\n‼️5% of wallet future"
msg5 = "🔼#Crv/USDT \n🟢LONG\n\n➡️Entry1: Market\n➡️Entry2: \n\n\n⛳️Targets : \n\n⛳️Target 1 : 1.221\n⛳️Target 2 : 1.245\n⛳️Target 3 : 1.274\n\n🚫 Stop : 1.145\n\n📊LV : 9X\n\n‼️3%  of wallet future\n#_کینگ‌کریپتو"
msg6 = "🔼#SOL/USDT 🟢LONG ➡️ Entry1: Market ➡️ Entry2: 234.4 ⛳️Target :242.8_246.9_253.1 🚫 Stop :232.7 📊LV : 15x ‼️3% of wallet future #_کینگ‌کریپتو"

king.findEntryTargets(msg=msg6)
# await rastad.test(msg=msg5, showPrint=True)


# messages = [msg, msg1, msg2, msg3, msg4, msg5, msg6]

# for i, msgTest in enumerate(messages):
#     res = await king.test(msg=msgTest, showPrint=True)
#     print_colored(f'number: {i+1} res: {res}', "#0ff" if res[0] else "#f00f00fc" )
#     print("****************************************************************\n\n")



['market', 234.4]

In [7]:
from Shared.updateEntryInMarketValue import updateEntryInMarketValue

await updateEntryInMarketValue(symbolName='CRV/USDT:USDT', entry_price=['market', 1.65], start_timestamp=1733643120000)

[[1733643120000, 1.1812, 1.1812, 1.1762, 1.1772, 55614.0]]


[1.1762, 1.65]

In [8]:
from Shared.findLongOrderStat import findLongOrderStat

await findLongOrderStat(symbolName='CRV/USDT:USDT', entry_price=[1.1762],  stop_loss=1.145, marketName=MarketValues.FUTURES, start_timestamp=1733643120000, max_day_wait=10, take_profit=[1.221, 1.245, 1.274] )

{'tps': [[1733663100000, 1.2032, 1.2212, 1.2012, 1.2202, 408222.0],
  [1733667540000, 1.2392, 1.2502, 1.2392, 1.2472, 164190.0]],
 'entry_reached': [[1733643120000, 1.1812, 1.1812, 1.1762, 1.1772, 55614.0]],
 'stop_loss_reached': [1733709900000, 1.1471, 1.1471, 1.1441, 1.1441, 56377.0],
 'break_reason': None}